# Exercise 04 : A/B-testing

In [1]:
import pandas as pd
import sqlite3

## create a connection to the database using the library sqlite3

In [2]:
connection = sqlite3.connect("../../data/checking-logs.sqlite")
cursor = connection.cursor()

## using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows

- time should have the values: after and before
- avg_diff contains the average delta among all the users for the time period before each of them made their first visit to the page and afterward
- only take into account the users that have observations before and after
- we still are not using the lab ’project1’

In [3]:
cursor.execute("DROP TABLE IF EXISTS test_results;")
connection.commit()

In [4]:
create_query = """
CREATE TABLE test_results (time TIMESTAMP CHECK(time IN ('before', 'after')), avg_diff FLOAT);
"""  
cursor.execute(create_query)
connection.commit()

In [5]:
# без CTE
# insert_data_query = """
#INSERT INTO test_results(time, avg_diff)
#SELECT 
#    time_periods.time,
#    AVG(CAST((unixepoch(t.first_commit_ts) - d.deadlines)/3600 AS INTEGER)) AS avg_diff
#FROM 
#    (SELECT 'before' AS time UNION ALL SELECT 'after') AS time_periods
#JOIN test t ON
#    (time_periods.time = 'before' AND t.first_commit_ts < t.first_view_ts) OR
#    (time_periods.time = 'after' AND t.first_commit_ts >= t.first_view_ts)
#JOIN deadlines d ON t.labname = d.labs
#WHERE 
#    t.uid IN (
#        SELECT t1.uid
#        FROM test t1
#        WHERE t1.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')
#        GROUP BY t1.uid
#        HAVING 
#            SUM(CASE WHEN t1.first_commit_ts < t1.first_view_ts THEN 1 ELSE 0 END) > 0 AND
#            SUM(CASE WHEN t1.first_commit_ts >= t1.first_view_ts THEN 1 ELSE 0 END) > 0
#    )
#GROUP BY time_periods.time
#ORDER BY time_periods.time = 'before' DESC
#"""
#cursor.execute(insert_data_query)
#connection.commit()
#test_results = pd.io.sql.read_sql("SELECT * FROM test_results", connection)
#test_results

In [6]:
#с СTE
# insert_data_query = """
#WITH 
#    time_periods AS (
#        SELECT 'before' AS time 
#        UNION ALL 
#        SELECT 'after' AS time
#    ),
#
#    valid_users AS (
#        SELECT t1.uid
#        FROM test t1
#        WHERE t1.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')
#        GROUP BY t1.uid
#        HAVING 
#            SUM(CASE WHEN t1.first_commit_ts < t1.first_view_ts THEN 1 ELSE 0 END) > 0 AND
#            SUM(CASE WHEN t1.first_commit_ts >= t1.first_view_ts THEN 1 ELSE 0 END) > 0
#    )
#
#INSERT INTO test_results(time, avg_diff)
#SELECT 
#    tp.time,
#    AVG(CAST((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24 AS INTEGER)) AS avg_diff
#FROM 
#    time_periods tp
#JOIN 
#    test t ON (tp.time = 'before' AND t.first_commit_ts < t.first_view_ts) OR
#              (tp.time = 'after' AND t.first_commit_ts >= t.first_view_ts)
#JOIN 
#    deadlines d ON t.labname = d.labs
#JOIN
#    valid_users v ON t.uid = v.uid
#GROUP BY 
#    tp.time
#ORDER BY 
#    tp.time = 'before' DESC;
#"""
##AVG(CAST((strftime("%s", t.first_commit_ts) - d.deadlines)/3600 AS INTEGER)) AS avg_diff
##AVG(CAST((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24 AS INTEGER)) 
##AVG((strftime('%s', t.first_commit_ts) - d.deadlines) / 3600.0) AS avg_diff
##AVG((strftime('%s', t.first_commit_ts) - strftime('%s', datetime(d.deadlines, 'unixepoch', 'localtime')))/ 3600.0) AS avg_diff -80 -107
##AVG((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch', 'localtime'))) * 24) AS avg_diff -80 -107
#cursor.execute(insert_data_query)
#connection.commit()
#test_results = pd.io.sql.read_sql("SELECT * FROM test_results", connection)
#test_results

In [7]:
#отдельный подсчет разницы для каждого юзера
insert_data_query = """
WITH 
valid_users AS (
    SELECT t1.uid
    FROM test t1
    WHERE t1.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')
    GROUP BY t1.uid
    HAVING 
        SUM(CASE WHEN t1.first_commit_ts < t1.first_view_ts THEN 1 ELSE 0 END) > 0 AND
        SUM(CASE WHEN t1.first_commit_ts >= t1.first_view_ts THEN 1 ELSE 0 END) > 0
),


time_differences AS (
    SELECT 
        CASE 
            WHEN t.first_commit_ts < t.first_view_ts THEN 'before'
            ELSE 'after'
        END AS time_period,
        CAST((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS hours_diff
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    JOIN valid_users v ON t.uid = v.uid
    WHERE t.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')
)

INSERT INTO test_results(time, avg_diff)
SELECT 
    time_period AS time,
    AVG(hours_diff) AS avg_diff
FROM time_differences
GROUP BY time_period
ORDER BY time_period = 'before' DESC;
"""
cursor.execute(insert_data_query)
connection.commit()
test_results = pd.io.sql.read_sql("SELECT * FROM test_results", connection)
test_results

,time,avg_diff
0,before,-60.5625
1,after,-104.6000


In [8]:
cursor.execute("DROP TABLE IF EXISTS control_results;")
connection.commit()

In [9]:
create_query = """
CREATE TABLE control_results (time TIMESTAMP CHECK(time IN ('before', 'after')), avg_diff INTEGER);
"""  
cursor.execute(create_query)
connection.commit()

In [10]:
# через отдельный подсчет
insert_data_query = """
WITH 
valid_users AS (
    SELECT c1.uid
    FROM control c1
    WHERE c1.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')
    GROUP BY c1.uid
    HAVING 
        SUM(CASE WHEN c1.first_commit_ts < c1.first_view_ts THEN 1 ELSE 0 END) > 0 AND
        SUM(CASE WHEN c1.first_commit_ts >= c1.first_view_ts THEN 1 ELSE 0 END) > 0
),


time_differences_control AS (
    SELECT 
        CASE 
            WHEN c.first_commit_ts < c.first_view_ts THEN 'before'
            ELSE 'after'
        END AS time_period,
        CAST((JULIANDAY(c.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS hours_diff
    FROM control c
    JOIN deadlines d ON c.labname = d.labs
    JOIN valid_users v ON c.uid = v.uid
    WHERE c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')
)

INSERT INTO control_results(time, avg_diff)
SELECT 
    time_period AS time,
    AVG(hours_diff) AS avg_diff
FROM time_differences_control
GROUP BY time_period
ORDER BY time_period = 'before' DESC;
"""
cursor.execute(insert_data_query)
connection.commit()
control_results = pd.io.sql.read_sql("SELECT * FROM control_results", connection)
control_results

,time,avg_diff
0,before,-99.464286
1,after,-117.636364


## close the connection

In [11]:
connection.close()

## have the answer: did the hypothesis turn out to be true and the page does affect the students’ behavior?

## В тестовой группе: до просмотра страницы новостей среднее время от первого коммита до дедлайна - 60 часов, после - 104 часа (дельта 44 часа), то есть те юзеры, которые просмотрели страницу новостей, действительно стали начинать проекты раньше.
## В контрольной группе выявлена похожая, но более слабая закономерность (99 часов и 117 часа, соответственно, дельта 18 часов). Это также может отражать естественную тенденцию пользователей делать первые коммиты ближе к дедлайну.